In [17]:

import pandas as pd


data = pd.read_csv("C:/Users/user/Downloads/FAOSTAT_data - FAOSTAT_data_en_12-29-2024.csv")  # Replace with your dataset path

# Pivot the dataframe to get separate columns for Area harvested, Yield, and Production
data_pivot = data.pivot_table(index=['Domain', 'Area Code (M49)', 'Area', 'Item Code (CPC)', 'Item', 'Year Code', 'Year'],
                              columns='Element',
                              values='Value',
                              aggfunc='first').reset_index()

# Flatten the multi-level columns
data_pivot.columns = [col if isinstance(col, str) else col[1] for col in data_pivot.columns]

# The dataframe now has separate columns for Area harvested, Yield, and Production
print(data_pivot.head())



                         Domain  Area Code (M49)         Area Item Code (CPC)  \
0  Crops and livestock products                4  Afghanistan               0   
1  Crops and livestock products                4  Afghanistan               0   
2  Crops and livestock products                4  Afghanistan             111   
3  Crops and livestock products                4  Afghanistan             111   
4  Crops and livestock products                4  Afghanistan             111   

     Item  Year Code  Year  Area harvested  Carcass Weight  Laying  \
0  Grapes       2019  2019             NaN             NaN     NaN   
1  Grapes       2021  2021             NaN             NaN     NaN   
2   Wheat       2019  2019       2334000.0             NaN     NaN   
3   Wheat       2020  2020       2668000.0             NaN     NaN   
4   Wheat       2021  2021       2046564.0             NaN     NaN   

   Milk Animals  Producing Animals/Slaughtered  Production  Stocks    Yield  
0           Na

In [18]:
data_pivot

Domain  Area Code (M49)         Area  \
0      Crops and livestock products                4  Afghanistan   
1      Crops and livestock products                4  Afghanistan   
2      Crops and livestock products                4  Afghanistan   
3      Crops and livestock products                4  Afghanistan   
4      Crops and livestock products                4  Afghanistan   
...                             ...              ...          ...   
92418  Crops and livestock products              894       Zambia   
92419  Crops and livestock products              894       Zambia   
92420  Crops and livestock products              894       Zambia   
92421  Crops and livestock products              894       Zambia   
92422  Crops and livestock products              894       Zambia   

      Item Code (CPC)                Item  Year Code  Year  Area harvested  \
0                   0              Grapes       2019  2019             NaN   
1                   0              Grapes       2021  2021             NaN   
2                 111               Wheat       2019  2019       2334000.0   
3                 111               Wheat       2020  2020       2668000.0   
4                 111               Wheat       2021  2021       2046564.0   
...               ...                 ...        ...   ...             ...   
92418            1708  Bambara beans, dry       2019  2019          6382.0   
92419            1708  Bambara beans, dry       2020  2020          6177.0   
92420            1708  Bambara beans, dry       2021  2021          6934.0   
92421            1708  Bambara beans, dry       2022  2022          6388.0   
92422            1708  Bambara beans, dry       2023  2023          6640.0   

       Carcass Weight  Laying  Milk Animals  Producing Animals/Slaughtered  \
0                 NaN     NaN           NaN                            NaN   
1                 NaN     NaN           NaN                            NaN   
2                 NaN     NaN           NaN                            NaN   
3                 NaN     NaN           NaN                            NaN   
4                 NaN     NaN           NaN                            NaN   
...               ...     ...           ...                            ...   
92418             NaN     NaN           NaN                            NaN   
92419             NaN     NaN           NaN                            NaN   
92420             NaN     NaN           NaN                            NaN   
92421             NaN     NaN           NaN                            NaN   
92422             NaN     NaN           NaN                            NaN   

       Production  Stocks    Yield  
0      1112927.00     NaN      NaN  
1             NaN     NaN  10727.9  
2      4890000.00     NaN   2095.1  
3      5185000.00     NaN   1943.4  
4      4017657.00     NaN   1963.1  
...           ...     ...      ...  
92418     6338.22     NaN    993.2  
92419     6388.82     NaN   1034.3  
92420     6757.73     NaN    974.5  
92421     5828.86     NaN    912.5  
92422     7294.46     NaN   1098.6  

[92423 rows x 15 columns]

In [20]:
# Check for missing values
print(data_pivot.isnull().sum())

# Option 1: Drop rows with missing values
data_pivot_cleaned = data_pivot.dropna()

# Option 2: Fill missing values with a specific value (e.g., 0 or mean)
data_pivot_cleaned = data_pivot.fillna(0)  # Fill with 0
# OR
data_pivot_cleaned = data_pivot.fillna(data_pivot.mean(numeric_only=True))  # Fill with column mean

Domain                               0
Area Code (M49)                      0
Area                                 0
Item Code (CPC)                      0
Item                                 0
Year Code                            0
Year                                 0
Area harvested                   44524
Carcass Weight                   86116
Laying                           91360
Milk Animals                     90315
Producing Animals/Slaughtered    74860
Production                       10481
Stocks                           83821
Yield                            41851
dtype: int64


In [21]:
# Check for duplicates
print(data_pivot.duplicated().sum())

# Drop duplicates
data_pivot_cleaned = data_pivot.drop_duplicates()

0


In [22]:
# Check data types
print(data_pivot.dtypes)

# Convert numeric columns to appropriate types
numeric_columns = ['Area harvested', 'Yield', 'Production']
data_pivot[numeric_columns] = data_pivot[numeric_columns].apply(pd.to_numeric, errors='coerce')

# Convert categorical columns to 'category'
categorical_columns = ['Domain', 'Area', 'Item']
data_pivot[categorical_columns] = data_pivot[categorical_columns].astype('category')

Domain                            object
Area Code (M49)                    int64
Area                              object
Item Code (CPC)                   object
Item                              object
Year Code                          int64
Year                               int64
Area harvested                   float64
Carcass Weight                   float64
Laying                           float64
Milk Animals                     float64
Producing Animals/Slaughtered    float64
Production                       float64
Stocks                           float64
Yield                            float64
dtype: object


In [23]:
from scipy.stats import zscore

# Calculate Z-scores for numeric columns
z_scores = data_pivot[numeric_columns].apply(zscore)

# Identify outliers (e.g., |Z-score| > 3)
outliers = (z_scores.abs() > 3).any(axis=1)

# Remove outliers
data_pivot_cleaned = data_pivot[~outliers]

In [24]:
from sklearn.preprocessing import MinMaxScaler

# Initialize the scaler
scaler = MinMaxScaler()

# Apply scaling to numeric columns
data_pivot[numeric_columns] = scaler.fit_transform(data_pivot[numeric_columns])

In [25]:
# One-hot encoding for categorical columns
data_pivot_encoded = pd.get_dummies(data_pivot, columns=categorical_columns, drop_first=True)

In [26]:
data_pivot_cleaned.to_csv("C:/Users/user/Downloads/FAOSTAT_cleaned_data.csv", index=False)

In [27]:
import pandas as pd

# Load the CSV file
file_path = "C:/Users/user/Downloads/FAOSTAT_cleaned_data.csv"  # Update if needed
data = pd.read_csv(file_path)

# Print missing value count before filling
print("\nMissing values before filling:\n", data.isnull().sum())

# Fill missing values
for col in data.columns:
    if data[col].dtype == "object":  # Categorical column
        data[col].fillna(data[col].mode()[0], inplace=True)  # Fill with mode
    else:  # Numerical column
        data[col].fillna(data[col].median(), inplace=True)  # Fill with median

# Print missing value count after filling
print("\nMissing values after filling:\n", data.isnull().sum())

# Save cleaned data
output_path = "C:/Users/user/Downloads/FAOSTAT_cleaned_data1.csv"
data.to_csv(output_path, index=False)
print(f"\nFilled data saved to {output_path}")



Missing values before filling:
 Domain                               0
Area Code (M49)                      0
Area                                 0
Item Code (CPC)                      0
Item                                 0
Year Code                            0
Year                                 0
Area harvested                   44524
Carcass Weight                   86116
Laying                           91360
Milk Animals                     90315
Producing Animals/Slaughtered    74860
Production                       10481
Stocks                           83821
Yield                            41851
dtype: int64

Missing values after filling:
 Domain                           0
Area Code (M49)                  0
Area                             0
Item Code (CPC)                  0
Item                             0
Year Code                        0
Year                             0
Area harvested                   0
Carcass Weight                   0
Laying                

C:\Users\user\AppData\Local\Temp\ipykernel_33072\3881415843.py:13: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  data[col].fillna(data[col].mode()[0], inplace=True)  # Fill with mode
C:\Users\user\AppData\Local\Temp\ipykernel_33072\3881415843.py:15: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behave


Filled data saved to C:/Users/user/Downloads/FAOSTAT_cleaned_data1.csv
